In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
result = model('sample.png')
result


image 1/1 c:\githome\wn7_dl_rep\sample.png: 640x608 3 persons, 43.3ms
Speed: 9.2ms preprocess, 43.3ms inference, 60.3ms postprocess per image at shape (1, 3, 640, 608)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [3]:
# 바운딩박스, 라벨 출력
result = model('red_panda.jpg')
result[0].boxes[0].conf[0]


image 1/1 c:\githome\wn7_dl_rep\red_panda.jpg: 352x640 3 bears, 33.6ms
Speed: 1.5ms preprocess, 33.6ms inference, 2.6ms postprocess per image at shape (1, 3, 352, 640)


tensor(0.9443, device='cuda:0')

In [4]:
import cv2 
img = cv2.imread('red_panda.jpg')
img

array([[[ 65, 154,  45],
        [ 72, 155,  52],
        [ 76, 149,  57],
        ...,
        [ 12,  16,  21],
        [  6,  10,  15],
        [ 28,  32,  37]],

       [[ 82, 171,  62],
        [ 74, 157,  54],
        [ 70, 143,  51],
        ...,
        [  6,  10,  15],
        [  0,   4,   9],
        [ 23,  27,  32]],

       [[ 76, 164,  58],
        [ 68, 151,  49],
        [ 67, 141,  47],
        ...,
        [  6,   9,  14],
        [  0,   3,   8],
        [ 22,  25,  30]],

       ...,

       [[112, 136,  94],
        [143, 171, 128],
        [157, 191, 150],
        ...,
        [200, 198, 198],
        [202, 200, 200],
        [204, 202, 202]],

       [[111, 132,  99],
        [124, 149, 115],
        [165, 198, 161],
        ...,
        [188, 186, 186],
        [183, 181, 181],
        [196, 194, 194]],

       [[135, 155, 126],
        [121, 145, 113],
        [165, 196, 163],
        ...,
        [178, 176, 176],
        [171, 169, 169],
        [183, 181, 181]]

In [6]:
for box in result[0].boxes:
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, f'{class_name} {conf:.4f}',
                (x1,y1-10), cv2.FONT_HERSHEY_PLAIN,
                1, (0,255,0),1)
    
# end-for
cv2.imshow('yolov8 object detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', img)

True

# Segmentation

In [19]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")
result = model('street.jpg')
result[0].show()


image 1/1 c:\githome\wn7_dl_rep\street.jpg: 512x640 17 persons, 3 buss, 2 backpacks, 1 handbag, 30.7ms
Speed: 3.2ms preprocess, 30.7ms inference, 32.9ms postprocess per image at shape (1, 3, 512, 640)


In [ ]:
import numpy as np

img_new = cv2.imread('street.jpg')
overlay = img_new.copy() # 마스크용

for box, mask in zip(result[0].boxes, result[0].masks):
    
    polygon = mask.xy[0].astype(np.int32)
    cv2.fillPoly(overlay, [polygon], (0, 255, 0))
    
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    
    cv2.rectangle(img_new, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img_new, f'{class_name} {conf:.4f}',
                (x1,y1-10), cv2.FONT_HERSHEY_PLAIN,
                1, (0,255,0),1)

# img + overlay combine and print
alpha = 0.5
final_img = cv2.addWeighted(overlay, alpha, img_new, 1-alpha, 0)

# 최종출력
cv2.imshow('yolov8 object segmentation', final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', final_img)

True

# 분류 (생략)
# 자세

In [25]:
pose_model = YOLO("yolov8n-pose.pt")
pose_result = pose_model('V.jpg')
img = cv2.imread('V.jpg')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x, y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(img, (int(x), int(y)), 3, (255, 0, 0), -1)
            
cv2.imshow('yolov8 object pose estimation', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('pose_result.jpg', img)


image 1/1 c:\githome\wn7_dl_rep\V.jpg: 640x640 1 person, 28.5ms
Speed: 5.1ms preprocess, 28.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


True

In [26]:
pose_model = YOLO("yolov8n-pose.pt")
pose_result = pose_model('RV.jpg')
img = cv2.imread('RV.jpg')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x, y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(img, (int(x), int(y)), 3, (255, 0, 0), -1)
            
cv2.imshow('yolov8 object pose estimation', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('pose_result.jpg', img)


image 1/1 c:\githome\wn7_dl_rep\RV.jpg: 448x640 3 persons, 33.9ms
Speed: 2.7ms preprocess, 33.9ms inference, 5.9ms postprocess per image at shape (1, 3, 448, 640)


True